1º Abrir servidor en mongo:
$mongod

In [ ]:
url = 'R:/Máster/SGDI/ProyectoStum/StumForYou-master/bbdd/usuariossinbliblioteca.csv'

In [ ]:
import csv
from pymongo import MongoClient

myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum']
mycol = mydb["usuarios"]

with open(url,encoding="utf8") as archivo:
    read = csv.reader(archivo)
    for row in read:
        print(row)
    ##datos=archivo.read()
##data=json.loads(datos)
##mycol.insert_many(data)

##for x in mycol.find():
  ##print(x)

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor()

mycursor.execute("SELECT id_juego,precio FROM juegos ORDER BY id_juego")
juegos = mycursor.fetchall()
mycursor.execute("SELECT id_dlc,precio,id_juego FROM dlcs ORDER BY id_dlc")
dlcs = mycursor.fetchall()
mycursor.execute("SELECT id_clave,id_dlc FROM claves_dlc ORDER BY id_clave")
claves_dlc = mycursor.fetchall()
mycursor.execute("SELECT id_clave,id_juego FROM claves_juegos ORDER BY id_clave")
claves_juego = mycursor.fetchall()
mycursor.execute("SELECT id_descuento,porcentaje_a_aplicar,id_juego,id_dlc FROM descuentos ORDER BY id_descuento")
descuentos = mycursor.fetchall()
mycursor.execute("SELECT id_proveedor FROM proveedor ORDER BY id_proveedor")
proveedor = mycursor.fetchall()

print(len(juegos))
print(len(dlcs))
print(len(claves_dlc))
print(len(claves_juego))
print(len(descuentos))
print(len(proveedor))


In [ ]:
import random
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor()

mycursor.execute("SELECT COUNT(*) FROM claves_juegos")
rango = mycursor.fetchone()[0] + 1
for i in range(1,rango):
    mycursor.execute("SELECT id_juego, fecha_anexion FROM claves_juegos WHERE id_clave = %s", (i,))
    datos = mycursor.fetchone()
    id_jue = datos[0]
    fecha = datos[1]
    
    mycursor.execute("SELECT precio FROM juegos WHERE id_juego = %s", (id_jue,))
    preciojuego = mycursor.fetchone()
    preciojuego = int(preciojuego[0] * 0.8)
    mycursor.execute("SELECT COUNT(*) FROM proveedor")
    longprov = mycursor.fetchone()[0] + 1
    prov = random.randrange(1,longprov)
    sql = "INSERT INTO transacciones_compra (precio_total, fecha_compra) VALUES (%s, %s)"
    val = (preciojuego, fecha)
    mycursor.execute(sql,val)
    mydb.commit()
    
    mycursor.execute("SELECT MAX(id_transaccion) FROM transacciones_compra")
    id_transac = mycursor.fetchone()[0]
    print((prov, id_transac, i, None, preciojuego))
    sql = "INSERT INTO compra_juegos (id_proveedor, id_transaccion, id_claves_juego, id_claves_dlc, precio) VALUES (%s, %s, %s, %s, %s)"
    val = (prov, id_transac, i, None, preciojuego)
    mycursor.execute(sql,val)
    mydb.commit()
        

In [ ]:
import random
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor()

mycursor.execute("SELECT COUNT(*) FROM claves_dlc")
rango = mycursor.fetchone()[0] + 1
for i in range(1,rango):
    mycursor.execute("SELECT id_dlc, fecha_anexion FROM claves_dlc WHERE id_clave = %s", (i,))
    datos = mycursor.fetchone()
    id_dlc = datos[0]
    fecha = datos[1]
    
    mycursor.execute("SELECT precio FROM dlcs WHERE id_dlc = %s", (id_dlc,))
    datos = mycursor.fetchone()
    preciodlc = int(datos[0] * 0.8)
    mycursor.execute("SELECT COUNT(*) FROM proveedor")
    longprov = mycursor.fetchone()[0] + 1
    prov = random.randrange(1,longprov)
    sql = "INSERT INTO transacciones_compra (precio_total, fecha_compra) VALUES (%s, %s)"
    val = (preciodlc, fecha)
    mycursor.execute(sql,val)
    mydb.commit()
    
    mycursor.execute("SELECT MAX(id_transaccion) FROM transacciones_compra")
    id_transac = mycursor.fetchone()[0]
    print((prov, id_transac, i, None, preciojuego))
    sql = "INSERT INTO compra_juegos (id_proveedor, id_transaccion, id_claves_juego, id_claves_dlc, precio) VALUES (%s, %s, %s, %s, %s)"
    val = (prov, id_transac, None, i, preciodlc)
    mycursor.execute(sql,val)
    mydb.commit()

In [ ]:
ALTER TABLE transacciones_compra AUTO_INCREMENT = 1
DELETE FROM transacciones_compra
DELETE FROM compra_juegos

import datetime as dt
from faker import Faker
fake = Faker()